In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import gtsa

from pathlib import Path
import psutil
import xarray as xr
import rasterio
import geopandas as gpd
import pandas as pd
import numpy as np
from sklearn import linear_model
import matplotlib.pyplot as plt
import hvplot.xarray

# Time series computations

Demonstrates memory-efficient per-pixel computations along the time axis.

#### Prerequesites
- Download DEM data with `00_download_dem_data.py` or `00_download_dem_data.ipynb`
- Prepare zarr stack with `01_create_stacks.py` or `01_create_stacks.ipynb`

## Start dask cluster

In [ ]:
workers = psutil.cpu_count(logical=True)-1
client = gtsa.io.dask_start_cluster(workers,
                                    ip_addres=None, # replace with address if working on remote machine
                                    port=':8787',
                                   )

## Read zarr stack

In [ ]:
data_dir = '../../data/dems/south-cascade/' # small test dataset
# data_dir = '../../data/dems/mount-baker' # large dataset

In [ ]:
zarr_fn = Path(data_dir, 'stack/stack.zarr').as_posix()
ds = xr.open_dataset(zarr_fn,chunks='auto',engine='zarr')

# optimize chunks and reload lazily
tc,yc,xc = gtsa.io.determine_optimal_chuck_size(ds,print_info = True) 
ds = xr.open_dataset(zarr_fn,chunks={'time': tc, 'y': yc, 'x':xc},engine='zarr')

# assign crs back
crs = rasterio.crs.CRS.from_epsg(32610)
ds = ds.rio.write_crs(crs)

In [ ]:
## optionally select a small window at the center of array for testing purposes
ds = gtsa.geospatial.extract_dataset_center_window(ds, size = 500)

In [ ]:
ds['band1'].isel(time=0).plot()

In [ ]:
ds['band1'] # each is dimensioned along the full time series

## Simple computations along time axis

### Count
Compute per-pixel count using built in xarray method

In [ ]:
%%time
count = ds['band1'].count(axis=0).compute()

In [ ]:
count.plot();

### NMAD
Compute per-pixel Normalized Median Absolute Deviation 

- This example highlights the use of a custom function

In [ ]:
def nmad(array):
    if np.all(~np.isfinite(array)):
        return np.nan
    else:
        return 1.4826 * np.nanmedian(np.abs(array - np.nanmedian(array)))

In [ ]:
result = xr.apply_ufunc(nmad, 
                        ds['band1'],
                        input_core_dims=[['time']],
                        vectorize=True, 
                        dask='parallelized',
                        output_dtypes=[float],
                       )

In [ ]:
%%time
array = result.compute()

In [ ]:
# create empty DataArray container
nmad_da = ds['band1'].isel(time=0).drop('time') 

# replace data values
nmad_da.values = array 

# give the data variable a name
nmad_da.name   = 'nmad' 

In [ ]:
nmad_da.plot(vmax=20);

## Linear regression

In [ ]:
# create a time series to make predictions on as decimal year floats
start = ds['time'].dt.strftime('%Y-%m-%d').values[0]
end = ds['time'].dt.strftime('%Y-%m-%d').values[-1]
prediction_time_series = gtsa.temporal.create_prediction_timeseries(start_date = start,
                                                                    end_date = end,
                                                                    dt ='Y')

# prepare time values as decimal year floats
times = [pd.to_datetime(j) for j in ds['band1'].time.values]
decyear_times = [gtsa.utils.date_time_to_decyear(i) for i in times] 
decyear_times = np.array(decyear_times)

ds['time'] = decyear_times

#### Example 1
Compute linear regression using a custom function with condition

In [ ]:
def custom_linreg(x,y, threshold = 2):
    mask = np.isfinite(y) # create mask for np.nan values
    if len(y[mask]) < threshold: # return np.nan if less than threshold
        return np.nan
    return np.polyfit(x[mask], y[mask], 1)[0]

In [ ]:
result = xr.apply_ufunc(
        custom_linreg, decyear_times , ds['band1'],
        input_core_dims=[['time'], ['time']],
        vectorize=True, 
        dask='parallelized',
        output_dtypes=[float],
        )

In [ ]:
custom_slope = result.compute()

In [ ]:
custom_slope.plot(vmin =-2, vmax = 2, cmap ='RdBu')

#### Example 2
Compute linear regression using built-in xarray polyfit function
- throws warnings when only a single data point is present in the time series

In [ ]:
result = ds['band1'].polyfit(dim='time', deg = 1).compute()

In [ ]:
slope = result['polyfit_coefficients'].sel(degree = 1)

In [ ]:
slope.plot(vmin =-2, vmax = 2, cmap ='RdBu')

#### Example 3

Compute linear regression using [Theil-Sen regressor](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.TheilSenRegressor.html)

- Using numpy for a linear regression is much faster than the external Theil-Sen function
- The Theil-Sen regression is more robust to outliers than a standard fit

In [ ]:
def ts_linreg(x,y, threshold = 2):
    mask = np.isfinite(y)
    if len(y[mask]) < threshold: 
        return np.nan 
    m = linear_model.TheilSenRegressor()
    return m.fit(x[mask][:, np.newaxis], y[mask]).coef_ # return slope

In [ ]:
result = xr.apply_ufunc(
        ts_linreg, decyear_times , ds['band1'],
        input_core_dims=[['time'], ['time']],
        vectorize=True, 
        dask='parallelized',
        output_dtypes=[float],
        )

In [ ]:
# ts_slope = result.compute()

In [ ]:
# ts_slope.plot(vmin =-2, vmax = 2, cmap ='RdBu')

## Fitting polynomials

In [ ]:
degree = 5
result = ds['band1'].polyfit(dim='time', deg = degree).compute()

In [ ]:
data = np.array(prediction_time_series)
coords = [np.array(prediction_time_series)]
dims = ['time']

In [ ]:
prediction_da = xr.DataArray(data,
                             coords,
                             dims = dims)
prediction_da.name = 'time'

In [ ]:
estimate = xr.polyval(prediction_da['time'], 
                      result.polyfit_coefficients)

estimate.name = 'elevation' 

In [ ]:
estimate.isel(time=1).plot()

In [ ]:
estimate.hvplot(groupby='time', 
                width=800,
                height=500,
                widget_type='scrubber', 
                widget_location='bottom', 
                clim=(1800,1900),
                cmap='viridis')

## Gaussian Process Regression

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (
    RBF,
    ConstantKernel,
    ExpSineSquared,
    PairwiseKernel,
    RationalQuadratic,
    WhiteKernel,
    Matern,
)

In [ ]:
k1 = 30.0 * Matern(length_scale=10.0, nu=1.5)
k2 = ConstantKernel(30) * ExpSineSquared(length_scale=1, periodicity=30)
k3 = ConstantKernel(30) * ExpSineSquared(length_scale=1, periodicity=1)

kernel = k1+k2+k3

In [ ]:
ds_result = gtsa.temporal.dask_apply_GPR(ds['band1'],
                                         'time', 
                                         kwargs={'times':ds['band1'].time.values,
                                                 'kernel': kernel,
                                                 'prediction_time_series' : prediction_time_series}
                                        ).compute()

In [ ]:
ds_result['mean_prediction'].isel(time=1).plot();

In [ ]:
ds_result['mean_prediction'].hvplot(groupby='time', 
                                    width=800,
                                    height=500,
                                    widget_type='scrubber', 
                                    widget_location='bottom', 
                                    clim=(1800,1900),
                                    cmap='viridis')